In [1]:
import os
import pandas as pd
import numpy as np
import requests
import gzip
import pgeocode
from meteostat import Point, Hourly
import datetime
import pathlib
import holidays
import zoneinfo

In [2]:
def getWeatherFilePath(load_area):
    filepath = "data/weather/" + load_area + ".csv"
    return filepath

def getPjmFilePath(year):
    filepath = "data/pjm/" + "hrl_load_metered_" + str(year) + ".csv"
    return filepath

def getPjmFreshFilePath():
    return "data/pjm/hrl_load_metered_fresh.csv"

def getCompleteDfFilePath(load_area):
    filepath = "data/complete_dfs/" + load_area + ".csv"
    return filepath

def getModelFilePath(load_area):
    filepath = "models/" + load_area + ".pkl"

def makeDirectories():
    base = pathlib.Path("data")
    subdirs = ["complete_dfs", "pjm", "weather"]
    for sub in subdirs:
        path = base / sub
        path.mkdir(parents=True, exist_ok=True)
        print(f"Created or already existed: {path}")
    path = pathlib.Path("models")
    path.mkdir(parents=True, exist_ok=True)
    print(f"Created or already existed: {path}")

makeDirectories()

Created or already existed: data/complete_dfs
Created or already existed: data/pjm
Created or already existed: data/weather
Created or already existed: models


In [3]:
def getLoadAreaToZips():

    # RTO is the *entire PJM footprint*, not a load zone. no meaningful ZIP.
    mymap = {
        # Atlantic City Electric (Southern New Jersey)
        'AECO': ['08401'],   # Atlantic City, NJ
    
        # American Electric Power - Appalachian Power (central and Southern West Virginia)
        'AEPAPT': ['25301'],   # Charleston, WV

        # American Electric Power - Indiana Michigan Power (northeast quadrant of indiana and southwest corner of michigan)
        'AEPIMP': ['46802'],   # Fort Wayne, IN

        # American Electric Power - Kentucky Power (eastern kentucky)
        'AEPKPT': ['41101'],   # Ashland, KY (Eastern Kentucky Power region)

        # American Electric Power - Ohio (central and southeast ohio)
        'AEPOPT': ['43215'],   # Columbus, OH

        # Allegheny Power (FirstEnergy West) serving MD/WV/PA panhandle
        'AP': ['21502'],     # Cumberland, MD

        # Baltimore Gas & Electric
        'BC': ['21201'],     # Baltimore, MD (city center)

        # Cleveland Electric Illuminating Company (FirstEnergy)
        'CE': ['44114'],     # Cleveland, OH (downtown)

        # Dayton Power & Light (AES Ohio)
        'DAY': ['45402'],    # Dayton, OH

        # Duke Energy Ohio/Kentucky load zone
        'DEOK': ['45202'],   # Cincinnati, OH

        # Dominion Virginia Power
        'DOM': ['23219'],    # Richmond, VA

        # Delmarva Power (Delaware & Eastern Shore MD)
        'DPLCO': ['19901'],  # Dover, DE

        # Duquesne Light
        'DUQ': ['15222'],    # Pittsburgh, PA

        # Easton Utilities (Maryland municipal)
        'EASTON': ['21601'], # Easton, MD

        # East Kentucky Power Cooperative
        'EKPC': ['40391'],   # Winchester, KY

        # Jersey Central Power & Light (FirstEnergy NJ)
        'JC': ['07728'],     # Freehold, NJ

        # Metropolitan Edison (FirstEnergy PA)
        'ME': ['19601'],     # Reading, PA

        # Ohio Edison (FirstEnergy OH)
        'OE': ['44308'],     # Akron, OH

        # Ohio Valley Electric Corporation
        'OVEC': ['45661'],   # Piketon, OH

        # Pennsylvania Power Company (FirstEnergy PA)
        'PAPWR': ['16101'],  # New Castle, PA

        # PECO (Philadelphia)
        'PE': ['19103'],     # Philadelphia, PA (Center City)

        # Potomac Electric Power Company (Washington DC + Montgomery Co MD)
        'PEPCO': ['20001'],  # Washington, DC

        # Potomac Edison (FirstEnergy MD/WV)
        'PLCO': ['21740'],   # Hagerstown, MD

        # Pennsylvania Electric Company (Penelec - FirstEnergy Northwest/Central PA)
        'PN': ['16601'],     # Altoona, PA

        # Public Service Electric & Gas (PSE&G NJ)
        'PS': ['07102'],     # Newark, NJ

        # Rockland Electric (Northern NJ / small NY portion)
        'RECO': ['07450'],   # Ridgewood, NJ

        # Southern Maryland Electric Cooperative
        'SMECO': ['20650'],  # Leonardtown, MD

        # UGI Electric (NE Pennsylvania, Luzerne County)
        'UGI': ['18702'],    # Wilkes-Barre, PA

        # VMEU = Virginia Municipal Electric Utility (multiple small cities)
        'VMEU': ['22801']    # Harrisonburg, VA (representative muni)
    }
    return mymap

# Returns the id of the load area (1-indexed)
def getLoadAreaId(load_area):
    load_areas = getLoadAreaToZips().keys()
    load_areas = sorted(load_areas)
    return load_areas.index(load_area)+1

In [4]:
def checkInconsistencies():
    load_area_to_zips = getLoadAreaToZips()
    for year in range(2016,2026):
        df = pd.read_csv(getPjmFilePath(year))
        #print(df['zone'].unique())
        #print(df['load_area'].unique())
        #print(len(df['load_area'].unique()))
        set1 = set(df['load_area'].unique())
        set2 = set(load_area_to_zips.keys())
        set2.add("RTO")
        # Note that years 2016 and 2017 do not match. Thus, we will start with 2018 inclusive
        if len(set1.symmetric_difference(set2)) > 0:
            print(str(year) + " did not match! The difference is: " + str(set1.symmetric_difference(set2)))
    
    print("Finished checking inconsistencies!")

checkInconsistencies()

2016 did not match! The difference is: {'VMEU', 'OVEC', 'AECO', 'AE'}
2017 did not match! The difference is: {'OVEC'}
Finished checking inconsistencies!


In [8]:
def getYears():
    return range(2018,2026)

def getWeatherDf(load_area, force_refresh=False):
    # Create a Nominatim geocoder instance for the desired country (For the USA, use 'us')
    nomi = pgeocode.Nominatim('us')
    years = getYears()
    # First check if we have the information cached
    file_path = getWeatherFilePath(load_area)
    if os.path.exists(file_path) and not force_refresh:
        temp = pd.read_csv(file_path)
        #print(temp.head())
        temp['time'] = pd.to_datetime(temp['time'])
        temp = temp.set_index('time')
        #print(temp.head())
        if temp.index.min().year == years[0] and temp.index.max().year == years[-1]:
            print("Cached weather file for " + load_area + " was found")
            return temp

    # Query the postal code
    load_area_to_zips = getLoadAreaToZips()
    zip_code = load_area_to_zips[load_area][0]
    location_data = nomi.query_postal_code(zip_code)
    latitude = float(location_data.latitude)
    longitude = float(location_data.longitude)
    #print(latitude)
    #print(longitude)

    location = Point(latitude, longitude)
    start = datetime.datetime(years[0], 1, 1)
    end   = datetime.datetime(years[-1], 12, 31)

    data = Hourly(location, start, end).fetch()
    #print(data.head())
    data.to_csv(getWeatherFilePath(load_area))
    print("Added new cached weather file for " + load_area + " using zip code:" + str(zip_code))
    return data

def getAllWeatherDfs():
    load_area_to_zips = getLoadAreaToZips()
    for key, value in load_area_to_zips.items():
        load_area = key
        getWeatherDf(load_area)

getAllWeatherDfs()
aeco_weather_df = getWeatherDf("AECO")
print(aeco_weather_df.head())

Cached weather file for AECO was found
Cached weather file for AEPAPT was found
Cached weather file for AEPIMP was found
Cached weather file for AEPKPT was found
Cached weather file for AEPOPT was found
Cached weather file for AP was found
Added new cached weather file for BC using zip code:21201
Cached weather file for CE was found
Cached weather file for DAY was found
Cached weather file for DEOK was found
Cached weather file for DOM was found
Cached weather file for DPLCO was found
Cached weather file for DUQ was found
Cached weather file for EASTON was found
Cached weather file for EKPC was found
Cached weather file for JC was found
Cached weather file for ME was found
Cached weather file for OE was found
Cached weather file for OVEC was found
Cached weather file for PAPWR was found
Cached weather file for PE was found
Cached weather file for PEPCO was found
Cached weather file for PLCO was found
Cached weather file for PN was found
Cached weather file for PS was found
Cached weath

In [14]:
def getUSHolidays():
    years = getYears()
    us_holidays = holidays.US(years=years)
    # Add black friday
    for year in years:
        thanksgiving = [day for day, name in us_holidays.items() if name == "Thanksgiving Day" and day.year == year][0]
        us_holidays[thanksgiving + datetime.timedelta(days=1)] = "Black Friday"
        us_holidays[thanksgiving - datetime.timedelta(days=1)] = "Thanksgiving Eve"

    return us_holidays

#print(us_holidays)
#print(us_holidays.keys())

In [15]:
def isHoliday(datetime_obj, holidays_map):
    return datetime_obj.date() in holidays_map

def isThanksgiving(datetime_obj, holidays_map):
    return holidays_map.get(datetime_obj.date()) == "Thanksgiving Day"
    
def isThanksgivingEve(datetime_obj, holidays_map):
    return holidays_map.get(datetime_obj.date()) == "Thanksgiving Eve"
    
def isBlackFriday(datetime_obj, holidays_map):
    return holidays_map.get(datetime_obj.date()) == "Black Friday"
    
def isWeekend(datetime_obj):
    weekno = datetime.datetime.today().weekday()
    if weekno < 5:
        return False
    return True

# Thanksgiving 2024 (Nov 28, 2024)
#test_dt = datetime.datetime(2024, 11, 29, 15, 0)  # 3pm on Thanksgiving Day
#print(isHoliday(test_dt, getUSHolidays()))
#print(isBlackFriday(test_dt, getUSHolidays()))

True
True


In [11]:
# Load PJM data

def getEnergyDf(load_area, force_refresh=False):
    years = getYears()
    load_area_to_zips = getLoadAreaToZips()
    if load_area not in load_area_to_zips.keys():
        print("Error: load_area(" + load_area + ") not found!")
        return -1

    ret_df = None
    for year in years:
        pjm_df = pd.read_csv(getPjmFilePath(year))
        pjm_df = pjm_df[pjm_df['load_area'] == load_area]
        
        if ret_df is None:
            ret_df = pjm_df
        else:
            ret_df = pd.concat([ret_df, pjm_df])

    # Note: 2025 is hardcoded, but whatever
    if 2025 in years:
        # first check if it is cached
        file_path = getPjmFreshFilePath()
        if not os.path.exists(file_path) or force_refresh:
            url = "https://raw.githubusercontent.com/SurtaiHan/stats604fa25proj4/refs/heads/main/data/pjm/hrl_load_metered_fresh.csv"
            out_path = pathlib.Path(getPjmFreshFilePath())
            response = requests.get(url)
            response.raise_for_status()  # raises if download failed
            out_path.write_bytes(response.content)
            print(f"Downloaded fresh pjm csv to: {out_path}")
        else:
             print("Cached pjm fresh file was found")
        
        print("Loading fresh data in addition to historical data")
        pjm_df = pd.read_csv(getPjmFreshFilePath())
        pjm_df = pjm_df[pjm_df['load_area'] == load_area]
        if ret_df is None:
            ret_df = pjm_df
        else:
            ret_df = pd.concat([ret_df, pjm_df])

    ret_df['datetime_beginning_utc'] = pd.to_datetime(
        ret_df['datetime_beginning_utc'], 
        #utc=True
    )
    ret_df = ret_df.set_index('datetime_beginning_utc')
    ret_df = ret_df.sort_index()   # Always a good idea after setting index

    return ret_df

aeco_energy_df = getEnergyDf("AECO")
print(aeco_energy_df.tail())

Cached pjm fresh file was found
Loading fresh data in addition to historical data
                       datetime_beginning_ept nerc_region mkt_region zone  \
datetime_beginning_utc                                                      
2025-11-10 00:00:00      11/9/2025 7:00:00 PM         RFC     MIDATL   AE   
2025-11-10 01:00:00      11/9/2025 8:00:00 PM         RFC     MIDATL   AE   
2025-11-10 02:00:00      11/9/2025 9:00:00 PM         RFC     MIDATL   AE   
2025-11-10 03:00:00     11/9/2025 10:00:00 PM         RFC     MIDATL   AE   
2025-11-10 04:00:00     11/9/2025 11:00:00 PM         RFC     MIDATL   AE   

                       load_area       mw  is_verified  
datetime_beginning_utc                                  
2025-11-10 00:00:00         AECO  980.590        False  
2025-11-10 01:00:00         AECO  952.221        False  
2025-11-10 02:00:00         AECO  908.209        False  
2025-11-10 03:00:00         AECO  855.014        False  
2025-11-10 04:00:00         AECO  79

In [18]:
# Last step is to augment the pjm data with:
# 1) weather at that current time  (we just use the ground truth weather)
# 2) previous day's weather
# 3) isHoliday, isThanksgiving, isThanksgivingEve, isBlackFriday, isWeekend
# 4) lagged loads (24 hrs ago, 168 hrs ago)

# The resulting dataframe could be viewed as X,y tuples
# in the sense that everything but 

# weather variables available:
# temp	dwpt	rhum	prcp	snow	wdir	wspd	wpgt	pres	tsun	coco

def add_features(energy_df, weather_df, dropna=True, outer=False):
    """
    df must contain at least:
    - load (float)
    - temp, dwpt, rhum, wspd, tsun, etc. (weather columns)
    - index is hourly timestamps (pd.DatetimeIndex)
    """

    """df = df.merge(
    weather_df[['temp']], # Only select 'temp', as the timestamp is the index
    left_on='datetime_beginning_utc',
    right_index=True,   # Tells pandas to use the index of weather_df (the DatetimeIndex)
    how='left'
    )"""
    df = energy_df.copy()

    if outer:
        df = df.join(weather_df[['temp']], how='outer')
    else:
        df = df.join(weather_df[['temp']], how='left')
    
    # --- Calendar Features ---
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    df['month'] = df.index.month
    df['is_weekend'] = df['dayofweek'].isin([5,6]).astype(int)

    # Holiday features (assuming you already defined isHoliday / isThanksgiving):
    us_holidays = getUSHolidays()
    df['is_holiday'] = df.index.to_series().apply(isHoliday, args=(us_holidays,))
    df['is_thanksgiving'] = df.index.to_series().apply(isThanksgiving, args=(us_holidays,))
    df['is_thanksgiving_eve'] = df.index.to_series().apply(isThanksgivingEve, args=(us_holidays,))
    df['is_black_friday'] = df.index.to_series().apply(isBlackFriday, args=(us_holidays,))

    # --- Degree Days ---
    df['HDD'] = (18 - df['temp']).clip(lower=0)
    df['CDD'] = (df['temp'] - 18).clip(lower=0)

    # --- Rolling Weather Averages ---
    df['temp_rolling_24'] = df['temp'].rolling('24h', min_periods=1).mean()
    df['temp_rolling_48'] = df['temp'].rolling('48h', min_periods=1).mean()
    df['temp_rolling_72'] = df['temp'].rolling('72h', min_periods=1).mean()
    # df['dwpt_rolling_24'] = df['dwpt'].rolling(24, min_periods=1).mean()

    # --- Lagged Weather Features ---
    # 24 48 72 96 120 144 168 192 216 240
    df['temp_lag_24'] = df['temp'].shift(freq='24h')
    df['temp_lag_48'] = df['temp'].shift(freq='48h')
    df['temp_lag_72'] = df['temp'].shift(freq='72h')
    df['temp_lag_96'] = df['temp'].shift(freq='96h')
    df['temp_lag_120'] = df['temp'].shift(freq='120h')
    df['temp_lag_168'] = df['temp'].shift(freq='168h')
    df['temp_lag_192'] = df['temp'].shift(freq='192h')
    df['temp_lag_216'] = df['temp'].shift(freq='216h')
    df['temp_lag_240'] = df['temp'].shift(freq='240h')

    # --- Lagged Load Features (Key Predictors) ---
    # 72 96 120 144 168 192 216 240
    df['mw_lag_72'] = df['mw'].shift(freq='72h')
    df['mw_lag_96'] = df['mw'].shift(freq='96h')
    df['mw_lag_120'] = df['mw'].shift(freq='120h')
    df['mw_lag_144'] = df['mw'].shift(freq='144h')
    df['mw_lag_168'] = df['mw'].shift(freq='168h')
    df['mw_lag_192'] = df['mw'].shift(freq='192h')
    df['mw_lag_216'] = df['mw'].shift(freq='216h')
    df['mw_lag_240'] = df['mw'].shift(freq='240h')

    print("Shape before dropna:", df.shape)
    # Drop rows where lag features are missing
    if dropna:
        df = df.dropna()
    print("Shape after dropna:", df.shape)

    return df

#df_augmented = add_features(aeco_energy_df, aeco_weather_df, False)
#print(df_augmented)

In [19]:
def getCompleteDf(load_area):
    # First check if we have the information cached
    file_path = getCompleteDfFilePath(load_area)
    if os.path.exists(file_path):
        temp = pd.read_csv(file_path)
        temp['datetime_beginning_utc'] = pd.to_datetime(temp['datetime_beginning_utc'])
        temp = temp.set_index('datetime_beginning_utc')
        temp = temp.sort_index()
        print("Cached complete file for " + load_area + " was found")
        return temp

    energy_df = getEnergyDf(load_area)
    weather_df = getWeatherDf(load_area)
    df_augmented = add_features(energy_df, weather_df)
    # load_area_to_complete_df[load_area] = df_augmented
    df_augmented.to_csv(getCompleteDfFilePath(load_area))
    print("Added new cached complete file for " + load_area)
    return df_augmented

def getAllCompleteDfs():
    load_area_2_complete_df = {}
    load_area_to_zips = getLoadAreaToZips()
    for load_area in load_area_to_zips.keys():
        load_area_2_complete_df[load_area] = getCompleteDf(load_area)
    return load_area_2_complete_df

# print(getCompleteDf("AECO").head())

load_area_to_complete_df = getAllCompleteDfs()

Cached complete file for AECO was found
Cached complete file for AEPAPT was found
Cached complete file for AEPIMP was found
Cached pjm fresh file was found
Loading fresh data in addition to historical data
Cached weather file for AEPKPT was found
Shape before dropna: (68856, 38)
Shape after dropna: (68051, 38)
Added new cached complete file for AEPKPT
Cached complete file for AEPOPT was found
Cached complete file for AP was found
Cached complete file for BC was found
Cached complete file for CE was found
Cached complete file for DAY was found
Cached complete file for DEOK was found
Cached complete file for DOM was found
Cached complete file for DPLCO was found
Cached complete file for DUQ was found
Cached complete file for EASTON was found
Cached complete file for EKPC was found
Cached complete file for JC was found
Cached complete file for ME was found
Cached complete file for OE was found
Cached complete file for OVEC was found
Cached complete file for PAPWR was found
Cached complete

In [21]:
# This function gets the EPT midnight of today, and expresses it as UTC
def getMidnightToday():
    #now = datetime.datetime.now()
    #return now.replace(hour=0, minute=0, second=0, microsecond=0)
    # Get current time in EPT (EST/EDT as appropriate)
    now_ept = datetime.datetime.now(zoneinfo.ZoneInfo("America/New_York"))
    midnight_ept = now_ept.replace(hour=0, minute=0, second=0, microsecond=0)
    midnight_utc = midnight_ept.astimezone(zoneinfo.ZoneInfo("UTC"))
    # Convert the aware datetime to a naive datetime
    midnight_utc = midnight_utc.replace(tzinfo=None)
    return midnight_utc

# This function gets the EPT midnight of tomorrow, and expresses it as UTC
def getMidnightTomorrow():
    #tmrw = datetime.datetime.now() + datetime.timedelta(days=1)
    #return tmrw.replace(hour=0, minute=0, second=0, microsecond=0)
    tmrw_ept = datetime.datetime.now(zoneinfo.ZoneInfo("America/New_York")) + datetime.timedelta(days=1)
    midnight_ept = tmrw_ept.replace(hour=0, minute=0, second=0, microsecond=0)
    midnight_utc = midnight_ept.astimezone(zoneinfo.ZoneInfo("UTC"))
    # Convert the aware datetime to a naive datetime
    midnight_utc = midnight_utc.replace(tzinfo=None)
    return midnight_utc

def getPredictionFeatures(load_area):

    energy_df = getEnergyDf(load_area)
    weather_df = getWeatherDf(load_area, force_refresh=True)
    prediction_df = add_features(energy_df, weather_df, dropna=False, outer=True)

    start = getMidnightToday()
    end = getMidnightTomorrow()
    prediction_df = prediction_df.loc[start:end]
    prediction_df["load_area"] = load_area
    return prediction_df

aeco_prediction_features = getPredictionFeatures("AECO")
print(aeco_prediction_features)

Cached pjm fresh file was found
Loading fresh data in addition to historical data
Added new cached weather file for AECO using zip code:08401
Shape before dropna: (69238, 38)
Shape after dropna: (69238, 38)
                    datetime_beginning_ept nerc_region mkt_region zone  \
2025-11-15 05:00:00                    NaN         NaN        NaN  NaN   
2025-11-15 06:00:00                    NaN         NaN        NaN  NaN   
2025-11-15 07:00:00                    NaN         NaN        NaN  NaN   
2025-11-15 08:00:00                    NaN         NaN        NaN  NaN   
2025-11-15 09:00:00                    NaN         NaN        NaN  NaN   
2025-11-15 10:00:00                    NaN         NaN        NaN  NaN   
2025-11-15 11:00:00                    NaN         NaN        NaN  NaN   
2025-11-15 12:00:00                    NaN         NaN        NaN  NaN   
2025-11-15 13:00:00                    NaN         NaN        NaN  NaN   
2025-11-15 14:00:00                    NaN         Na

In [43]:
print(getMidnightToday())
print(getMidnightTomorrow())

2025-11-15 05:00:00
2025-11-16 05:00:00
